# Initialising Your Agent as an Issuing Authority

This template walks you through the basic steps you need to take to configure your agent as an issuing authority on the Sovrin StagingNet. If using a different network you will need to update this template. The steps include:

* Writing your DID to the Sovrin StagingNet
* Accepting the Transaction Author Agreement
* Authoring schema to the ledger
* Authoring credential definitions for the schema this agent intends to issue
* Persisting Identifiers for use throughout the playground.

It is recommended that this initialisation notebook be run **once**. If you are following the default docker-compose services then your agents wallet storage will be persisted in a postgres database as long as you run `./manage stop` rather than `./manage down`. 



### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://bob-agent:3021 and an api key of adminApiKey


## Write DID to the Public Ledger

Note: if defined a ACAPY_WALLET_SEED value for your agent then this function will return a DID, but this DID still needs to be written to the ledger. If you did not define a seed you will need to create a DID first.

In [3]:
public_did_response = await agent_controller.wallet.get_public_did()

In [4]:
if public_did_response["result"]:
    did_obj = public_did_response["result"]
else:
    create_did_response = await agent_controller.wallet.create_did()
    did_obj = create_did_response['result']
print("DID", did_obj)

DID {'did': 'MH4Kr2vuxWmoWzD3rAhCa7', 'verkey': 'C3zVTCJJWyPBobRsNYHyij6zRo2jdFJVGmZAtF2TY3ut', 'posture': 'wallet_only', 'key_type': 'ed25519', 'method': 'sov'}


## Copy DID and Verkey across to Endorser Who Will Write it to the Ledger

In [5]:
print(f"did='{did_obj['did']}'")
print(f"verkey='{did_obj['verkey']}'")

did='MH4Kr2vuxWmoWzD3rAhCa7'
verkey='C3zVTCJJWyPBobRsNYHyij6zRo2jdFJVGmZAtF2TY3ut'


## Accept Transaction Author Agreement

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [6]:
taa_response = await agent_controller.ledger.get_taa()
TAA = taa_response['result']['taa_record']
TAA['mechanism'] = "service_agreement"
await agent_controller.ledger.accept_taa(TAA)

{}

## Assign Agent Public DID if Not Set

Will only be ran if ACAPY_WALLET_SEED not initially set.

In [7]:
if did_obj["posture"] != "public":
    response = await agent_controller.wallet.assign_public_did(did_obj["did"])
print("Successfully intialised agent with Public DID : ", did_obj["did"])

Successfully intialised agent with Public DID :  MH4Kr2vuxWmoWzD3rAhCa7


## Start Webhook Server

In [8]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


In [9]:
listeners = []

In [10]:
def transaction_handler(payload):
    print("Transaction Handler")
    print(payload)
    
transaction_listener = {
    "topic": "transaction",
    "handler": transaction_handler

}

listeners.append(transaction_listener)

In [11]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")
    if state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

In [12]:
agent_controller.register_listeners(listeners)

## Connect With Transaction Endorser

In [13]:
invitation = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '3c3394f6-8705-4318-9858-a792ce7a471a', 'label': 'Alice', 'serviceEndpoint': 'http://975f9ceaca5e.ngrok.io', 'recipientKeys': ['FAKi9XEBgyFMA3L4eFXaVtgZri3cTSRQLSVBr9gi9zN3']}


In [14]:
auto_accept="false"
alias=None

invite_response = await agent_controller.connections.receive_invitation(invitation, alias, auto_accept)
connection_id = invite_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  e6b70eda-c12c-4c84-b7c1-661a978efc80
State :  invitation
Routing State :  none
Their Role :  inviter
----------------------------------------------------------


In [15]:
# Label for the connection
my_label = None
# Endpoint you expect to recieve messages at
my_endpoint = None

accept_response = await agent_controller.connections.accept_invitation(connection_id, my_label, my_endpoint)

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  e6b70eda-c12c-4c84-b7c1-661a978efc80
State :  request
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  e6b70eda-c12c-4c84-b7c1-661a978efc80
State :  response
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  e6b70eda-c12c-4c84-b7c1-661a978efc80
State :  active
Routing State :  none
Their Role :  inviter
----------------------------------------------------------
Connection ID: e6b70eda-c12c-4c84-b7c1-661a978efc80 is now active.


## Send Trust Ping

Once connection moves to response state one agent, either inviter or invitee needs to send a trust ping.

Note: you may not need to run this cell. It depends one of the agents has the ACAPY_AUTO_PING_CONNECTION=true flag set.

In [ ]:
comment = "Some Optional Comment"
message = await agent_controller.messaging.trust_ping(connection_id, comment)

## Writing Schema

Write as many schema to the ledger as you like. Be sure to store each schema_id, you will need these when it comes to authoring credential defintition transactions and issuing credentials against this schema.

Uncomment and copy the below cell as many times as you need. Be sure to update any arugments surrounded by <> with your own details.

In [16]:
# # Define you schema name - must be unique on the ledger
schema_name = "some schema"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["data"]



In [17]:
## THIS SHOULD FAIL AS PUBLIC DID DOES NOT HAVE ENDORSER ROLE

In [18]:
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]

## Configure Transaction Endorser

In [19]:
transaction_my_job = "TRANSACTION_AUTHOR"
response =  await agent_controller.transactions.set_endorser_role(connection_id, transaction_my_job)

In [21]:
endorser_did="7BVkD5QLC2MMLFGq2uoeU8"
endorser_name="My Endorser"
response = await agent_controller.transactions.set_endorser_info(connection_id, endorser_did,endorser_name)

In [22]:

create_txn_for_endorser = "true"
response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version, connection_id, create_txn_for_endorser)


KeyError: 'schema_id'

In [24]:
transaction_id = response['txn']['transaction_id']
print(transaction_id)

b925e0e7-2e7f-4ae9-9eea-65fcacf00bab


In [29]:
endorser_write_transaction = "true"
body = {
  "expires_time": "2021-09-29T05:22:19Z"
}
response = await agent_controller.transactions.create_request(transaction_id, endorser_write_transaction, body)

In [31]:
transaction = await agent_controller.transactions.get_by_id(transaction_id)
print(transaction)

{'thread_id': 'e8b080ce-1dc2-4694-b1ab-42d8f26437af', 'updated_at': '2021-07-09 16:45:39.773209Z', 'connection_id': 'e6b70eda-c12c-4c84-b7c1-661a978efc80', 'signature_request': [{'context': 'did:sov', 'method': 'add-signature', 'signature_type': '<requested signature type>', 'signer_goal_code': 'transaction.endorse', 'author_goal_code': 'transaction.ledger.write'}], 'signature_response': [{'message_id': 'f0614c99-554e-4d6a-9add-d3f4a5a58a2e', 'context': 'did:sov', 'method': 'add-signature', 'signer_goal_code': 'transaction.endorse', 'signature_type': '<requested signature type>', 'signature': {'7BVkD5QLC2MMLFGq2uoeU8': '{"endorser":"7BVkD5QLC2MMLFGq2uoeU8","identifier":"MH4Kr2vuxWmoWzD3rAhCa7","operation":{"data":{"attr_names":["data"],"name":"some schema","version":"0.0.1"},"type":"101"},"protocolVersion":2,"reqId":1625848632333366752,"signatures":{"7BVkD5QLC2MMLFGq2uoeU8":"qyXjJu8HH7ShREreewBYyKcBr5h3MZbbmHPYMmBEy5gvaT41NVBEtGKFZUfWn3NPxNXoj27Pndh8NmcDG8dPXE8","MH4Kr2vuxWmoWzD3rAhCa7

In [33]:
await agent_controller.schema.get_created_schema()

{'schema_ids': ['MH4Kr2vuxWmoWzD3rAhCa7:2:some schema:0.0.1',
  'MH4Kr2vuxWmoWzD3rAhCa7:2:some schema:0.0.1']}

## Writing Credential Definitions

For all schema you intend to issue credentials against your agent must author a credential definition transaction to the public ledger. This specifies the public cryptographic material your agent will use to sign all credentials issued against a specific schema. 

Again uncomment and copy this cell as often as you need. Remebering to update the arguments in <> to specify your schema identifiers. Store each credential definition identifier in a unique variable.

In [1]:
# Tag and group specific credential definitions
tag = "default"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
# Note: ACAPY_TAILS_SERVER_BASE_URL env var must be properly configured
support_revocation = False

# cred_def_response = await agent_controller.definitions.write_cred_def(<SCHEMA ID>, tag, support_revocation)
# <YOUR CRED DEF ID> = cred_def_response["credential_definition_id"]

## Persist Identifiers for use throughout other business logic notebooks associated with this agent

The schema_id and cred_def_id value pairs are required whenever issuing credentials, and also can be used to constrain acceptable proof requests. In a real application these values might be stored in environment variables or most likely in a database. For notebooks we have found it easier to store as string values in a cell and then load these values into the jupyter store so that they can be fetched across multiple notebooks.

As such you are recommended to print out each of the schema and cred def identifiers used by your agent and copy them across to your **main** business logic notebook where you should store them in a variable and save them to the jupyter store. Remember, you should only be running this notebook once so having this logic in here will not be useful.


In [ ]:
print(f"schema_id='{<YOUR SCHEMA ID>}'")
print(f"cred_def_id='{<YOUR CRED DEF ID>}'")

## Terminate Controller


In [27]:
await agent_controller.terminate()